# Data Preperation
The resulting data from this notebook will combine the positional data (latitude, longitude, town) of the ChemDataForJeffOlson.csv and all of the tables from the Land use survey.
## Merge all tables from the land use survey
Here we run through the files from the land use survey. All files have the same colum names and thus we just need to append them all. I created a dataframe from the first file, then ran through the rest of them turning each into a dataframe then appending it to the first.

In [1]:
import os
import re
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
# set the path to the folder with all the tables
land_use_folder = 'assets/Tables'

# set the path to the chem data file
chem_data_file_path = 'assets/ChemDataForJeffOlson.csv'

# set the path to the population data (Optional)
use_population = True
population_path = 'assets/HS-STAT-Population-of-Vermont-towns-1930-2019.xls'

# set the path to the characteristics data
characteristics_path = 'assets/Characteristic.csv'

# set the save path to the resulting cleaned chem data
chem_data_save_path = 'assets/chem_data_merged.csv'

# set the save path to the resulting durvey data file
survay_save_path = 'assets/combined_tables.csv'

In [3]:
# combine all tables from Table Folder in assets folder

def merge_tables_folder(tables_folder):
    # get the first file as data frame to append to
    file_1 = os.listdir(tables_folder)[0]
    tables_df = pd.read_excel(os.path.join(tables_folder, file_1))
    
    # add file name to df
    tables_df['from_file'] = file_1
    
    # run through the rest of the files and append them to the data frame
    for file in os.listdir(tables_folder)[1:]:
        df = pd.read_excel(os.path.join(tables_folder, file))
        df['from_file'] = file
        tables_df = pd.concat((tables_df, df), axis=0)
    
    # drop OBJECTID column and reset index
    tables_df = tables_df.drop('OBJECTID', axis=1).reset_index(drop=True)
    return tables_df

In [4]:
combined_tables_df = merge_tables_folder(land_use_folder)

In [5]:
combined_tables_df.sample(5)

,Description,Shape_Length,Shape_Area,TREE_CANOPY_acres,GRASS_SHRUBS_acres,BARE_SOIL_acres,WATER_acres,BUILDINGS_acres,ROADS_acres,OTHER_PAVED_acres,RAILROADS_acres,Ag_Crops_acres,Ag_Hay_acres,Ag_Pasture_acres,Ag_Total_acres,Imp_Bare_Soil_acres,Imp_Buildings_acres,Imp_Other_Paved_acres,Imp_Road_acres,Imp_Railroad_acres,Imp_Total_acres,Shrub_Shrubs_acres,Shrub_Total_acres,TC_Coniferous_acres,TC_Deciduous_acres,TC_Total_acres,Wet_Emergent_acres,Wet_Forested_acres,Wet_Scrub_Shrub_acres,Wet_Total_acres,from_file
346,MILES_Buffer250ftWaterbody,10855.694248,3.950464e+05,57.329040,23.343886,0.607137,5.079058,2.601213,2.890573,5.757794,0.000000,0.000000,0.000000,0.000000,0.000000,0.687833,2.817030,6.920777,4.539928,0.00000,14.965569,0.000000,0.000000,20.190305,37.426668,57.616973,4.571986,4.946705,1.930201,11.448891,AOIs_MILES.xlsx
125,Watershed_SOUTHMARLBR,7560.728663,1.061841e+06,258.859847,2.626108,0.182672,0.449546,0.092541,0.083830,0.094703,0.000000,0.000000,0.000000,0.000000,0.000000,0.210477,0.173460,0.203516,2.099397,0.00000,2.686849,0.000000,0.000000,82.161365,176.745233,258.906598,0.679557,29.689523,0.000000,30.369080,AOIs_SOUTHMARLBR.xls
32,Waterbody100ft_CRYSTALBARTON,21425.369129,3.230344e+05,40.132199,23.599454,0.964883,5.875416,1.256529,2.708827,3.112658,2.164516,0.000000,0.000000,0.000000,0.000000,1.142903,1.413879,3.400538,2.793151,2.33304,11.083512,3.665897,3.665897,8.571034,31.753771,40.324805,7.322188,5.667986,1.756313,14.746487,AOIs_CRYSTALBARTON.xls
535,WINONA_Buffer100ftWBFL,45597.110617,1.216806e+06,119.963485,158.048729,0.610411,18.173646,0.175012,1.072436,2.603745,0.000000,28.351377,28.048842,2.023828,58.424046,0.600182,0.285157,2.621646,1.480298,0.00000,4.987283,82.544288,82.544288,32.837400,87.239058,120.076458,93.345800,25.267570,2.394888,121.008258,AOIs_WINONA.xlsx
345,MILES_Buffer100ftWBFL,33571.468505,8.872185e+05,158.943743,40.909221,1.334861,12.208779,1.781812,1.288838,2.730325,0.000000,0.000000,0.046209,0.000000,0.046209,2.268529,1.955045,3.142849,1.430235,0.00000,8.796658,13.497389,13.497389,56.126873,103.139706,159.266579,22.009749,37.685543,10.777552,70.472844,AOIs_MILES.xlsx


## Split the Description column to get LakeIDs
The Description column has the LakeID attached to a description. depending on the lake they may be LakeID then description or description then LakeID. the function below splits on an underscore and takes the part of the newly created list that is all uppercase as the LakeID and leaves the rest as the description.

In [6]:
# now I want to solit the description column in to two columns 
# one corresponds to the title area and the other corresponds to the description
# depending on the folder they are in different orders

def split_description(df):
    df['Description'] = df['Description'].str.split('_')
    df['LakeID'] = df['Description'].apply(lambda row: row[0] if row[0].isupper() else row[1])
    df['Description'] = df['Description'].apply(lambda row: row[0] if not row[0].isupper() else row[1])
    return df

In [7]:
combined_tables_df = split_description(combined_tables_df)
combined_tables_df.sample(5)

,Description,Shape_Length,Shape_Area,TREE_CANOPY_acres,GRASS_SHRUBS_acres,BARE_SOIL_acres,WATER_acres,BUILDINGS_acres,ROADS_acres,OTHER_PAVED_acres,RAILROADS_acres,Ag_Crops_acres,Ag_Hay_acres,Ag_Pasture_acres,Ag_Total_acres,Imp_Bare_Soil_acres,Imp_Buildings_acres,Imp_Other_Paved_acres,Imp_Road_acres,Imp_Railroad_acres,Imp_Total_acres,Shrub_Shrubs_acres,Shrub_Total_acres,TC_Coniferous_acres,TC_Deciduous_acres,TC_Total_acres,Wet_Emergent_acres,Wet_Forested_acres,Wet_Scrub_Shrub_acres,Wet_Total_acres,from_file,LakeID
615,Waterbody100ft,25848.860649,3.824119e+05,31.584838,40.896742,0.132819,5.114579,6.042582,4.563535,6.143772,0.0,0.000000,0.436235,0.000000,0.436235,0.218915,6.374069,6.585005,5.517186,0.0,18.695175,3.129802,3.129802,18.878959,12.875689,31.754647,14.162417,3.902440,1.434831,19.499688,AOIs_JOESDANVLL.xls,JOESDANVLL
570,Buffer100ftWBFL,37085.090528,1.051215e+06,178.221022,61.730350,0.000000,19.082498,0.176865,0.375785,0.132634,0.0,0.000000,0.000000,0.000000,0.000000,0.032960,0.174920,0.147822,0.542286,0.0,0.897987,18.804740,18.804740,62.514744,116.132380,178.647123,47.856550,72.707353,21.810496,142.374399,AOIs_TURTLEHEAD.xlsx,TURTLEHEAD
577,Watershed,26255.948768,1.240012e+07,2442.955335,378.234604,2.166864,205.276486,5.650426,12.592841,17.253798,0.0,1.002242,99.779605,6.964394,107.746240,2.433793,6.159827,19.122303,25.899482,0.0,53.615405,50.193579,50.193579,707.954787,1740.035570,2447.990357,69.912424,189.203787,45.701129,304.817339,AOIs_MIRROR.xlsx,MIRROR
319,Waterbody100ft,20761.204950,3.146472e+05,49.994469,17.703220,0.382024,3.648876,2.706109,0.085251,3.215701,0.0,0.000000,0.000000,0.000000,0.000000,0.518078,2.979960,3.968096,0.125383,0.0,7.591517,2.978658,2.978658,22.400158,27.779810,50.179968,2.977293,8.448367,1.208428,12.634089,AOIs_GROTON.xls,GROTON
616,Buffer100ftWBFL,221827.631536,6.607924e+06,1126.417888,346.215356,0.891308,128.688639,7.120578,12.670618,10.526055,0.0,0.000000,61.997425,0.050623,62.048048,1.392318,7.460474,11.343402,15.028948,0.0,35.225142,50.376926,50.376926,623.949458,505.139049,1129.088507,168.329790,523.203015,91.527321,783.060126,AOIs_JOESDANVLL.xls,JOESDANVLL


## Merge relevant columns from the chem data and population datasets to survey data
For now I will just take the 'LakeID', 'Lat', 'Long', 'Town' from the chem data dataset. for other use later i will also extract the measurements when I know what year the land use survey was done. I will need to fix the LakeIDs in the combined_tables_df dataframe so that they match up with the LakeIDs in the chem dataset. This is the column that i will join on.

In [8]:
chem_data_df = pd.read_csv(chem_data_file_path)
chem_data_df.sample(5)

,LakeID,LakeStationNo,LakeStationType,Lat,Long,Town,ProjectID,VisitDate,VisitNumber,StartTime,CollectionMethodID,Depth,ActivityCategory,CharacteristicID,Symbol,Result,Calcs,ProjRemark,RemarkCode,DepthStratumCode
142457,LITTLE (WOODFD),1,Pelagic,42.92500,-73.06560,Woodford,AcidLake,1981-04-29 00:00:00,1,1500.0,TygonHose,2.00,Reg,TNOX,NaN,0.67,Y,NaN,NaN,NaN
98511,GROUT,1,Pelagic,43.04451,-72.94366,Stratton,AcidLake,1982-08-17 00:00:00,1,1115.0,TygonHose,6.00,Reg,GranAlk,NaN,1.61,Y,NaN,NaN,NaN
46565,CHARLESTON,1,Pelagic,44.89250,-72.05420,Charleston,SpringTP,2013-05-02 00:00:00,1,1126.0,Hydrolab,1.01,Reg,DO%,NaN,99.40,Y,NaN,NaN,NaN
62813,DUNMORE,1,Pelagic,43.91220,-73.07640,Salisbury,LakeAsmt,1995-09-07 00:00:00,1,1329.0,Thermister,10.00,Reg,TempC,NaN,12.80,Y,NaN,NaN,NaN
103222,HALLS,1,Pelagic,44.08610,-72.12390,Newbury,SpringTP,1990-04-20 00:00:00,1,NaN,Secchi,NaN,Reg,Secchi,NaN,3.30,Y,NaN,NaN,NaN


In [9]:
if use_population:    
    population = pd.read_excel(population_path, skiprows=4, index_col='CTC')
    # lets take the relevent columns from the population dataset and melt them so we ca easily join the dataframes
    population = population[['NAME'] + list(population.columns[-32:])]
    population.rename(columns={'NAME': 'Town'}, inplace=True)
    population.sample(5)

In [10]:
# now lets convert town names in the chem dataset to uppercase to match the town names in the population data set
chem_data_df['Town'] = chem_data_df['Town'].str.upper()
# we also need to extract the year from the chem data set so that we can join the population data
# to the chem data by year and Town
chem_data_df['VisitDate'] = pd.to_datetime(chem_data_df['VisitDate'])
chem_data_df['year'] = chem_data_df['VisitDate'].dt.year

In [11]:
chem_to_add = chem_data_df[['LakeID', 'Lat', 'Long', 'Town']].copy()
chem_to_add.drop_duplicates(inplace=True)

# get all lake ids that are also in usage survey and fix them
for lake_id in np.unique(chem_to_add['LakeID']):
    lake_id_fixed = re.sub('[()\s;]', '', lake_id)
    index = combined_tables_df[combined_tables_df['LakeID'] == lake_id_fixed].index
    combined_tables_df.loc[index, 'LakeID'] = lake_id


In [12]:
# merge with the chem_to_add data frame
new_data_df = combined_tables_df.merge(chem_to_add, how='left', on='LakeID')

# reorder columns
cols = list(new_data_df.columns)

to_front = ['LakeID', 'Description', 'Lat', 'Long', 'Town']
for col in to_front:
    cols.remove(col)

cols = to_front + cols
new_data_df = new_data_df.reindex(columns=cols)

new_data_df.head(5)

,LakeID,Description,Lat,Long,Town,Shape_Length,Shape_Area,TREE_CANOPY_acres,GRASS_SHRUBS_acres,BARE_SOIL_acres,WATER_acres,BUILDINGS_acres,ROADS_acres,OTHER_PAVED_acres,RAILROADS_acres,Ag_Crops_acres,Ag_Hay_acres,Ag_Pasture_acres,Ag_Total_acres,Imp_Bare_Soil_acres,Imp_Buildings_acres,Imp_Other_Paved_acres,Imp_Road_acres,Imp_Railroad_acres,Imp_Total_acres,Shrub_Shrubs_acres,Shrub_Total_acres,TC_Coniferous_acres,TC_Deciduous_acres,TC_Total_acres,Wet_Emergent_acres,Wet_Forested_acres,Wet_Scrub_Shrub_acres,Wet_Total_acres,from_file
0,HORSE,Watershed,44.6169,-72.2108,GREENSBORO,9262.633620,2.107941e+06,468.319079,45.532062,0.034904,3.322265,0.043058,3.534652,0.097545,0.0,0.0,0.0,0.0,0.0,0.067911,0.042648,0.116058,5.195567,0.0,5.422184,0.946536,0.946536,210.794069,258.350785,469.144853,14.288349,77.086939,5.588561,96.963848,AOIs_HORSE.xls
1,HORSE,Flowline100ft,44.6169,-72.2108,GREENSBORO,1500.131136,4.176164e+04,4.545867,3.129955,0.000000,2.491621,0.000000,0.150858,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.151524,0.0,0.151524,0.279845,0.279845,2.970089,1.629099,4.599187,3.341299,2.996817,0.908847,7.246963,AOIs_HORSE.xls
2,HORSE,Waterbody100ft,44.6169,-72.2108,GREENSBORO,4447.434860,6.269552e+04,6.338120,8.104550,0.000000,0.834165,0.022425,0.093591,0.096186,0.0,0.0,0.0,0.0,0.0,0.000000,0.022314,0.114463,0.177650,0.0,0.314428,0.011841,0.011841,5.278762,1.092946,6.371707,5.845567,2.616113,2.488963,10.950643,AOIs_HORSE.xls
3,HORSE,Buffer100ftWBFL,44.6169,-72.2108,GREENSBORO,5715.509700,1.020007e+05,10.868234,10.706442,0.000000,3.262898,0.022425,0.244449,0.096186,0.0,0.0,0.0,0.0,0.0,0.000000,0.022314,0.114463,0.329174,0.0,0.465952,0.291685,0.291685,8.234746,2.720674,10.955420,8.581423,5.612931,3.397809,17.592163,AOIs_HORSE.xls
4,HORSE,Buffer250ftWaterbody,44.6169,-72.2108,GREENSBORO,4647.798280,1.545388e+05,23.735053,12.987901,0.000000,0.874999,0.022425,0.466102,0.097545,0.0,0.0,0.0,0.0,0.0,0.000000,0.022314,0.116058,0.614319,0.0,0.752692,0.604020,0.604020,16.834859,7.006856,23.841715,9.646655,8.976926,3.505087,22.128668,AOIs_HORSE.xls


In [13]:
# now since the land use survey was conducted from 2013 - 2016 I will merge the population data for 2016 joining on the town names
if use_population:    
    pop2016 = population[['Town', 2016]].copy()
    pop2016.rename(columns={2016: '2016_population'}, inplace=True)
    new_data_df = new_data_df.merge(pop2016, how='left', on='Town')
    print('Final survey data DataFrame shape: ', new_data_df.shape)
    new_data_df.sample(5)

Final survey data DataFrame shape:  (773, 36)


## Save the survey dataframe
Lets save the resulting dataframe as a .csv for later use

In [14]:
# save the data frame as a .csv
new_data_df.to_csv(survay_save_path)

## Merge the population dataset and the chem data dataset
Here we will take the population dataset and melt it so that the years are also in the rows sp that we can merge on both town and year data.

In [15]:
# melt the population dataframe
if use_population:
    population_1 = population.melt(id_vars='Town', var_name='year', value_name='population')
    chem_data_df = chem_data_df.merge(population_1, how='left', on=['Town', 'year'])
    chem_data_df.sample(5)

In [16]:
# Now so that we have measurements 

characteristics = pd.read_csv(characteristics_path)
characteristics.sample(5)

,CharacteristicID,CharacteristicName,UnitCode,SampleFraction
84,TMg,Total Magnesium,mg/l,Total
82,TK,Total Potassium,mg/l,Total
76,TCu,Total Copper,ug/l,Total
37,DOP,Dissolved Ortho Phosphorus,ug/l,Dissolved
92,TNO3,Total Nitrate Nitrogen,mg/l,Total


In [17]:
# Merge characteristics df
chem_data_df_final = chem_data_df.merge(characteristics, how='left', on='CharacteristicID')
print('Final chem data Data frame shape: ', chem_data_df_final.shape)
chem_data_df_final.sample(5)

Final chem data Data frame shape:  (284912, 25)


,LakeID,LakeStationNo,LakeStationType,Lat,Long,Town,ProjectID,VisitDate,VisitNumber,StartTime,CollectionMethodID,Depth,ActivityCategory,CharacteristicID,Symbol,Result,Calcs,ProjRemark,RemarkCode,DepthStratumCode,year,population,CharacteristicName,UnitCode,SampleFraction
106768,HARDWOOD,1,Pelagic,44.46869,-72.49955,ELMORE,AcidLake,1994-10-12,1,1032.0,PlasticKemm,1.00,Reg,DMg,NaN,0.476,Y,NaN,NaN,NaN,1994,687.0,Dissolved Magnesium,mg/l,Dissolved
37460,CARMI,1,Pelagic,44.97393,-72.87549,FRANKLIN,TMDL,2019-06-19,1,1020.0,BottleGrab,0.20,Reg,TCa,NaN,13.700,Y,NaN,NaN,NaN,2019,1425.0,Total Calcium,mg/l,Total
122044,HORTONIA,1,Pelagic,43.75530,-73.20220,HUBBARDTON,SpringTP,1998-04-09,1,1015.0,Hydrolab,8.60,Reg,DO,NaN,9.870,Y,NaN,NaN,NaN,1998,720.0,Dissolved Oxygen,mg/l,NaN
270102,VAIL,1,Pelagic,44.70496,-72.07220,SUTTON,SpringTP,2013-05-13,1,1302.0,Hydrolab,4.55,Reg,TempC,NaN,6.950,Y,NaN,NaN,NaN,2013,1026.0,Temperature,deg C,NaN
2489,BALD HILL,1,Pelagic,44.73940,-71.97780,WESTMORE,SpringTP,2005-05-09,1,1206.0,Hydrolab,6.00,Reg,DO,NaN,10.020,Y,NaN,NaN,NaN,2005,335.0,Dissolved Oxygen,mg/l,NaN


## Save the chem data DataFrame
Lets save the resulting dataframe as a .csv for later use

In [18]:
chem_data_df_final.to_csv(chem_data_save_path)

## Documantation

In [19]:
%load_ext watermark
%watermark --iversions

re    : 2.2.1
pandas: 1.3.4
numpy : 1.21.2

